# Import packages

In [1]:
import numpy as np
import time
from copy import deepcopy

# PQR

In [2]:
def PQR(matrix, n, m, r = 0, eps = 0):

    if not r or r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))
    P = np.arange(m)

    tol = eps + 1
    i = 0

    while i < r and tol > eps:

        norms = np.array([np.linalg.norm(matrix[:, j]) for j in range(i, m)])
        tol = np.sqrt(np.sum(norms**2))
        args = np.argsort(norms)[::-1]
        args += i

        P[[i, args[0]]] = P[[args[0], i]]
        R[:, [i, args[0]]]  = R[:, [args[0], i]]
        matrix[:, [i, args[0]]] = matrix[:, [args[0], i]]

        R[i, i] = norms[args[0] - i]
        Q[:, i] = matrix[:, i] / R[i, i]

        for j in range(i + 1, m):
            R[i, j] = matrix[:, j] @ Q[:, i]
            matrix[:, j] -= R[i, j] * Q[:, i]

        i += 1

    return Q, R, P, i

# MaxVol

In [3]:
def maxvol(mat, n, m, r, eps = 0):

    if not r or r > min(n, m):
          r = min(n, m)

    rows = np.random.permutation(n)
    B = mat[rows[:r], :]
    _, _, cols, rank = PQR(B, r, m, r, eps)


    rev_rows = np.zeros(n, dtype=int)
    rev_cols = np.zeros(m, dtype=int)

    for i in range(n):
        rev_rows[rows[i]] = i
    for i in range(m):
        rev_cols[cols[i]] = i

    matrix = mat[np.ix_(rows, cols)]

    R = np.arange(r)

    C = matrix[:, :r]
    A = C[:r, :]

    inv_A = np.linalg.pinv(A)

    prod = C @ inv_A

    max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    iter = 0

    while np.abs(prod[max_val_ind]) > 1 and max_val_ind[0] >= r and iter < r * r:

        iter += 1

        vec = prod[max_val_ind[0], :].copy()
        vec[max_val_ind[1]] -= 1
        vec /= prod[max_val_ind]

        prod -= prod[:, max_val_ind[1]].reshape((-1, 1)) @ vec.reshape((1, -1))

        R[max_val_ind[1]] = max_val_ind[0]

        max_val_ind = np.unravel_index(np.argmax(np.abs(prod), axis=None), prod.shape)

    prod = prod[rev_rows, :]
    R = matrix[np.ix_(R, rev_cols)]

    return prod, R

# Cross Approximation

In [4]:
def cross_approx(matrix, n, m, r = 0, eps = 0):

    if not r and r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))

    vec = np.arange(m)

    i = 0

    max_el = eps + 1

    while i < r and max_el * np.sqrt((n - i) * (m - i)) > eps:

        ind = np.random.randint(m - i)
        col = vec[ind]

        row = np.argmax(np.abs(matrix[:, col] - Q[:, :i] @ R[:i, col]), axis=None)
        col = np.argmax(np.abs(matrix[row, :] - Q[row, :i] @ R[:i, :]), axis=None)

        val_col = matrix[:, col] - Q[:, :i] @ R[:i, col]
        row = np.argmax(np.abs(val_col), axis=None)

        val_row = matrix[row, :] - Q[row, :i] @ R[:i, :]

        Q[:, i] = val_col / val_col[row]
        R[i, :] = val_row

        max_el = np.abs(matrix[row, col])

        ind = np.where(vec == col)[0][0]

        vec = np.delete(vec, ind)

        i += 1

    return Q, R, i

# Cross Approximation Dzheltkov

In [153]:
def cross_approx_dzh(matrix, n, m, r = 0, eps = 0):

    if not r and r > min(m, n):
        r = min(m, n)

    Q = np.zeros((n, r))
    R = np.zeros((r, m))

    vec = np.arange(m)

    i = 0
    max_el = eps + 1

    rows = np.zeros((r, m))
    ind_rows = np.ones(r, dtype=int) * n
    free_row = 0
    cols = np.zeros((n, r))
    ind_cols = np.ones(r, dtype=int) * m
    free_col = 0

    while i < r and max_el * np.sqrt((n - i) * (m - i)) > eps:

        ind = np.random.randint(m - i)
        col = vec[ind]

        if col in ind_cols:
            val = np.where(ind_cols == col)[0][0]
            row = np.argmax(np.abs(cols[:, val]), axis=None)
        else:
            cols[:, free_col] = matrix[:, col] - Q[:, :i] @ R[:i, col]
            ind_cols[free_col] = col
            row = np.argmax(np.abs(cols[:,free_col]), axis=None)
            free_col += 1

        if row in ind_rows:
            val = np.where(ind_rows == row)[0][0]
            col = np.argmax(np.abs(rows[val, :]), axis=None)
        else:
            rows[free_row, :] = matrix[row, :] - Q[row, :i] @ R[:i, :]
            ind_rows[free_row] = row
            col = np.argmax(np.abs(rows[free_row, :]), axis=None)
            free_row += 1

        if col in ind_cols:
            val = np.where(ind_cols == col)[0][0]
            val_col = cols[:, val]
            row = np.argmax(np.abs(val_col), axis=None)
        else:
            val_col = matrix[:, col] - Q[:, :i] @ R[:i, col]
            row = np.argmax(np.abs(val_col), axis=None)

        val_row = matrix[row, :] - Q[row, :i] @ R[:i, :]

        Q[:, i] = val_col / val_col[row]
        R[i, :] = val_row

        if col in ind_cols:
            val = np.where(ind_cols == col)[0][0]
            ind_cols = np.delete(ind_cols, val)
            cols = np.delete(cols, val, 1)
            free_col -= 1

        if row in ind_rows:
            val = np.where(ind_rows == row)[0][0]
            ind_rows = np.delete(ind_rows, val)
            rows = np.delete(rows, val, 0)
            free_row -= 1

        rows[:free_row, :] -= Q[ind_rows[:free_row], i].reshape((-1, 1)) @ R[i, :].reshape((1, -1))
        cols[:, :free_col] -= Q[:, i].reshape((-1, 1)) @ R[i, ind_cols[:free_col]].reshape((1, -1))

        max_el = np.abs(val_col[row])
        ind = np.where(vec == col)[0][0]
        vec = np.delete(vec, ind)
        i += 1

    return Q, R, i

# Create matrix

In [7]:
n = 1000
m = 1000
mat_cr = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        mat_cr[i, j] = 1 / (i + j + 1)

In [162]:
matrix = mat_cr.copy()
mat_copy = mat_cr.copy()

# Run Cross Approximation

In [161]:
start_time = time.time()
Q, R, rank = cross_approx(matrix, n, m, 20)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.00612950325012207
1.5798740392664099e-10


# Run Cross Approximation Dzheltkov

In [163]:
start_time = time.time()
Q, R, rank = cross_approx_dzh(matrix, n, m, 20)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.010917186737060547
1.0267451585140687e-10


# Run PQR

In [212]:
start_time = time.time()
Q, R, P, rank = PQR(matrix, n, m, 10)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy[:, P]) / np.linalg.norm(mat_copy))

time = 0.18038082122802734
2.3628691531130358e-05


# Run MaxVol

In [231]:
start_time = time.time()
Q, R = maxvol(matrix, n, m, 20)
end_time = time.time()
print(f"time = {end_time - start_time}")

print(np.linalg.norm(Q @ R - mat_copy) / np.linalg.norm(mat_copy))

time = 0.19413137435913086
0.0007071955025624902
